In [10]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model

In [11]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
    include_top = False, 
    weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [12]:
# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense  (3, activation='softmax')(x)           

model = Model( pre_trained_model.input, x)


In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.95,
    beta_2=0.999,
    epsilon=1e-07
),
    loss='CategoricalCrossentropy',
    metrics=['AUC',"accuracy"]
)

In [14]:
train_datagen= ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'datasetjpgrgb/train',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=128,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'datasetjpgrgb/val',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=1000,
        class_mode='categorical')

Found 30000 images belonging to 3 classes.
Found 7500 images belonging to 3 classes.


In [15]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [17]:
model.fit(
        train_generator,
        steps_per_epoch=80,
        epochs=500,
        validation_data=validation_generator,
        validation_steps=3)

Epoch 1/500
80/80 [==============================] - ETA: 0s - loss: 1.0838 - auc: 0.5755 - accuracy: 0.3925

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[1000,64,72,72] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_3/conv2d_96/Conv2D (defined at <ipython-input-16-e97581f3cf15>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[assert_greater_equal/Assert/AssertGuard/pivot_f/_3/_23]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[1000,64,72,72] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_3/conv2d_96/Conv2D (defined at <ipython-input-16-e97581f3cf15>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add